## Overview

Sometimes the MOH adds facilities that aren't in the project database. This script makes it easy to add these in using a payload that looks like this:

```
{
    "name": "testadfasdf",
    "shortName": "testadfasdf",
    "openingDate": "2023-02-01T07:00:00.000Z",
    "parent": {
        "id": "kuKm18r4bfg"
    }
}
```

In [ ]:

# Load up environment
from dotenv import load_dotenv
import os

load_dotenv()

print('HANDY_DHIS2_USER_SOURCE: {}'.format(os.getenv('HANDY_DHIS2_USER_SOURCE')))
print('HANDY_DHIS2_PASSWORD_SOURCE: {}'.format(os.getenv('HANDY_DHIS2_PASSWORD_SOURCE')))
print('HANDY_DHIS2_HOST_SOURCE: {}'.format(os.getenv('HANDY_DHIS2_HOST_SOURCE')))

print('HANDY_DHIS2_USER_DEST: {}'.format(os.getenv('HANDY_DHIS2_USER_DEST')))
print('HANDY_DHIS2_PASSWORD_DEST: {}'.format(os.getenv('HANDY_DHIS2_PASSWORD_DEST')))
print('HANDY_DHIS2_HOST_DEST: {}'.format(os.getenv('HANDY_DHIS2_HOST_DEST')))

In [ ]:
import json
# This is an metadata export of Organization Units

f = open('/Users/apreston/Data/Senegal/DHIS2/senegal-ous.json')
all_ous = json.load(f)


In [ ]:


import json

ous = []
# Opening JSON file
f = open('ou-conflicts.json') # this is the response payload with errors that orgs don't exist
data = json.load(f)
for conflict in data['response']['conflicts']:

    # go through the whole darn list
    for ou in all_ous['organisationUnits']:
        if ou['id'] == conflict['object']:
            myou = {}
            myou['id']= ou['id']
            myou['name']=ou['name']
            myou['parent'] = ou['parent']
            if 'geometry' in ou:
                myou['geometry'] = ou['geometry']
            ous.append(myou)
    
print('found {} new ous to add'.format( len(ous)) )

In [ ]:
#add the missing orgs one at a time

import requests
import os

for n in ous:

    response = {}
    response['id'] = n['id']
    response['name'] = n['name']
    response['shortName'] = n['name']
    response['openingDate'] = '2020-02-01T07:00:00.000Z'
    response['parent'] = { 'id' : n['parent']}

    try:
        r = requests.post(os.getenv('HANDY_DHIS2_HOST_DEST')+'/api/organisationUnits', json=response,
              auth=(os.getenv('HANDY_DHIS2_USER_DEST'),os.getenv('HANDY_DHIS2_PASSWORD_DEST'))  )
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        print(e)

    print(r.text)




In [ ]:
# Here is the curl command if you want
print('curl --user '+os.getenv('HANDY_DHIS2_USER_DEST')+':'+os.getenv('HANDY_DHIS2_PASSWORD_DEST')+' '+os.getenv('HANDY_DHIS2_HOST_DEST') + '/api/organisationUnits --header "Content-Type: application/json" --data @covid-data-to-import.json')